In [1]:
import numpy as np
import pandas as pd
import os
from topicpy.hsbmpy import get_max_available_L

In [2]:
directory=f"/home/jovyan/work/phd/datasets/tcga/COAD/mirna_allsamples/"
os.chdir(directory)

In [3]:
L = get_max_available_L(directory, "trisbm")

In [4]:
os.listdir()

['trisbm',
 'files.dat.dvc',
 'mainTable_all.csv.dvc',
 'mainTable_hv.csv',
 'mainTable_fpkm.csv.dvc',
 'files_miRNA.dat.dvc',
 'mainTable_mirna.csv',
 'mainTable_all.csv',
 'files_fpkm.dat.dvc',
 '.gitignore',
 'files.dat',
 'files_with_isella.dat',
 'mainTable_fpkm.csv',
 'mainTable_mirna.csv.dvc',
 'files_isella.csv',
 'mainTable_hv.csv.dvc',
 'files_manifest.dat.dvc',
 'files_miRNA.dat',
 'files_manifest.dat',
 'files_fpkm.dat']

In [5]:
df_files = pd.read_csv("files.dat", index_col=0)
df_files.head(2)

,cases.0.samples.0.portions.0.analytes.0.aliquots.0.submitter_id.1,level_0,index,cases.0.demographic.days_to_birth,cases.0.demographic.days_to_death,cases.0.demographic.gender,cases.0.demographic.vital_status,cases.0.diagnoses.0.age_at_diagnosis,cases.0.diagnoses.0.days_to_last_follow_up,cases.0.diagnoses.0.last_known_disease_status,...,cases.0.demographic.gender.1,cases.0.demographic.vital_status.1,cases.0.diagnoses.0.age_at_diagnosis.1,cases.0.diagnoses.0.days_to_last_follow_up.1,cases.0.diagnoses.0.last_known_disease_status.1,cases.0.diagnoses.0.tumor_stage.1,cases.0.project.project_id.1,cancer.type,Subtype_other,Subtype_Selected
cases.0.submitter_id,,,,,,,,,,,,,,,,,,,,,
TCGA-AA-3516,TCGA-AA-3516-11A-01R-A32Z-07,23a4119f-8c45-4391-b7d0-39d981cea5b6.mirbase21...,9e8b528b-1172-4c07-a09b-ebb23cf2310c.FPKM.txt.gz,-27303.0,396.0,female,Dead,27303.0,0.0,not reported,...,female,Dead,27303.0,0.0,not reported,stage iii,TCGA-COAD,NaN,NaN,NaN
TCGA-AA-3854,TCGA-AA-3854-01A-01R-0905-07,e9c6a3b0-2ee2-4d0f-9f35-bcb00dfd7932.mirbase21...,5697212f-b3fd-479f-84b0-ec0aae54534a.FPKM.txt.gz,-26022.0,NaN,female,Alive,26022.0,1096.0,not reported,...,female,Alive,26022.0,1096.0,not reported,stage i,TCGA-COAD,COAD,GS,GI.GS


In [6]:
df_isella = pd.read_csv("files_isella.csv", index_col=0)
df_isella.index = ["-".join(sample.split(".")[:3]) for sample in df_isella.index]
df_isella.head(2)

,CCMS Class,CRCA Class,CCS Class,Subgroup,Tumor Purity estimated by Absolute
TCGA-A6-2674,4,5,3,SSM,0.25
TCGA-A6-2685,4,5,3,SSM,0.36


In [7]:
df_files = df_files.join(df_isella, how="inner")
df_files.to_csv("files_with_isella.dat")

In [8]:
import graph_tool as gt
import sys
sys.path.append("trisbm")

In [12]:
%load_ext autoreload
%autoreload 
from trisbm.trisbm import trisbm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
df = pd.read_csv("mainTable_all.csv", index_col=0)
df = df.loc[:,df.columns.isin(df_files.index)]
df.head(2)

,TCGA-AA-3516,TCGA-AA-3854,TCGA-A6-2679,TCGA-AA-3494,TCGA-AA-A00D,TCGA-AA-3971,TCGA-A6-4107,TCGA-AA-3877,TCGA-CM-4746,TCGA-AA-3673,...,TCGA-AA-3502,TCGA-AA-3667,TCGA-AA-3986,TCGA-CK-4951,TCGA-CA-5256,TCGA-AA-3852,TCGA-AA-3812,TCGA-CM-4752,TCGA-AA-3679,TCGA-AA-3952
ENSG00000000419,17.11,21.44,20.27,38.88,43.33,53.78,40.38,27.99,41.87,46.98,...,21.57,31.35,35.74,31.92,115.44,17.34,9.32,73.93,61.95,45.50
ENSG00000000457,2.91,2.96,2.55,1.93,2.14,2.81,2.56,2.40,1.55,2.88,...,2.62,1.13,2.67,1.79,1.68,1.91,1.13,1.91,2.43,2.27


In [13]:
model = trisbm()

In [14]:
model.make_graph(df, lambda gene: 1 if "ENSG" in gene else 2)

In [15]:
model.save_graph("graph.xml.gz")

In [ ]:
model.fit()

    l=1  Current bracket: (3, 3, 3) (76.01268864650585, 76.01268864650585, 76.01268864650585)
    l=1  Current bracket: (3, 3, 3) (76.01268864650585, 76.01268864650585, 76.01268864650585)
    l=1  Bisect at B = 3 with S = 76.01268864650585
    l=1  Best result: B = 3, S = 76.01268864650585
level 1 : rejected replacement (3, 1) -> (3, 3) , dS: 2.890371757792309
    l=1  Current bracket: (3, 3, 3) (76.01268864650585, 76.01268864650585, 76.01268864650585)
    l=1  Current bracket: (3, 3, 3) (76.01268864650585, 76.01268864650585, 76.01268864650585)
    l=1  Bisect at B = 3 with S = 76.01268864650585
    l=1  Best result: B = 3, S = 76.01268864650585
level 1 : rejected insert 3 , dS: 5.780743515584618
    l=0      B: 10882 <- 17647    shrinking 17647 -> 13575
    l=0      B: 10882 <- 17647    B=13575  niter:     1  count:    0  breaks:  0  min_S: 2141986.1  max_S: 2151882.6  S: 2141986.1  ΔS:     -9896.53  moves:  1648 
    l=0      B: 10882 <- 17647    B=13575  niter:     2  count:    0  b